# Post Training Dynamic Quantization

In Dynamic quantization we calculate the scale factor for the activations dynamically based on the data observed at runtime.

The model parameter are already known during conversion and so they are convered and stored in INT8 before inference.

In [1]:
# importing the necessary libraries
import torch
from torch.quantization import quantize
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from tqdm import tqdm
from fastprogress.fastprogress import master_bar, progress_bar
from pathlib import Path
import os
import time


def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    size = os.path.getsize("temp_delme.p")/1e3
    print('Size (KB):', size)
    os.remove('temp_delme.p')
    return size


# Define the network:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def train_model(model, train_dl, valid_dl, criterion, optimizer):
    mb = master_bar(range(5))
    for epoch in mb:
        running_loss = 0.0
        correct_train, total_train = 0, 0
        # Progress bar for training batches
        pb = progress_bar(train_dl, parent=mb)

        for inputs, labels in pb:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Compute training accuracy
            preds = torch.argmax(outputs, dim=1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

            mb.child.comment = f"Train Loss: {loss.item():.4f}"

        # Compute average train loss & accuracy
        avg_train_loss = running_loss / len(train_dl)
        train_accuracy = correct_train / total_train * 100

        # Validation Phase (No Gradients)
        val_loss = 0.0
        correct_val, total_val = 0, 0

        with torch.no_grad():
            # Progress bar for validation
            pb = progress_bar(valid_dl, parent=mb)

            for inputs, labels in pb:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Compute validation accuracy
                preds = torch.argmax(outputs, dim=1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

                mb.child.comment = f"Valid Loss: {loss.item():.4f}"

        # Compute average validation loss & accuracy
        avg_val_loss = val_loss / len(valid_dl)
        val_accuracy = correct_val / total_val * 100

        # Write epoch summary
        mb.write(f"Epoch {epoch+1}: "
                 f"Train Loss={avg_train_loss:.4f}, Train Acc={train_accuracy:.2f}% | "
                 f"Val Loss={avg_val_loss: .4f}, Val Acc={val_accuracy: .2f}")


# evaluate
def evaluate(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            predicted = outputs.argmax(dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print("test acc", correct / total)
    return correct/total

/home/sonu/code/model_quantization/.venv/lib/python3.11/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [2]:
# config
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
batch_size = 32
epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# dataset and dataloader
train_ds = torchvision.datasets.CIFAR10(
    root='./data/cifar', train=True, transform=tfms, download=True)
test_ds = torchvision.datasets.CIFAR10(
    root='./data/cifar', train=False, transform=tfms, download=True)

train_ds, valid_ds = random_split(train_ds, [40000, 10000])
print(f"train ds length {len(train_ds)}")
print(f"test ds length {len(test_ds)}")
# train_ds = Subset(train_ds, range(100))
# valid_ds = Subset(train_ds, range(100))
# test_ds = Subset(train_ds, range(100))

train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(
    valid_ds, batch_size=batch_size, shuffle=False)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=batch_size, shuffle=False)

train ds length 40000
test ds length 10000


In [4]:
# defining loss function and optimizer
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# saving the model
model_path = "model.pkl"
if Path(model_path).exists():
    model = Net()
    model.load_state_dict(torch.load(model_path))
    print('model loaded successfully')
else:
    train_model(model, train_dl, valid_dl, criterion, optimizer)
    torch.save(model.state_dict(), 'model.pkl')

Epoch 1: Train Loss=2.2590, Train Acc=14.86% | Val Loss= 2.0955, Val Acc= 23.00                     
Epoch 2: Train Loss=1.9649, Train Acc=28.44% | Val Loss= 1.8663, Val Acc= 31.34                     
Epoch 3: Train Loss=1.7319, Train Acc=36.78% | Val Loss= 1.6596, Val Acc= 39.01                     
Epoch 4: Train Loss=1.5727, Train Acc=42.14% | Val Loss= 1.5431, Val Acc= 43.80                     
Epoch 5: Train Loss=1.4839, Train Acc=46.12% | Val Loss= 1.4437, Val Acc= 47.26                     


In [5]:
print('Weights before quantization')
print(model.fc1.weight)
print(model.fc1.weight.dtype)
print('Size of the model before quantization')
o_size = print_size_of_model(model)
print('Accuracy of the model before quantization: ')
tik = time.time()
o_acc = evaluate(model, test_dl)
tok = time.time()
o_time = tok-tik

Weights before quantization
Parameter containing:
tensor([[ 0.0104, -0.0451, -0.0420,  ..., -0.0018, -0.0146,  0.0187],
        [-0.0603,  0.0163,  0.0094,  ...,  0.0270, -0.0356,  0.0184],
        [-0.0088,  0.0259,  0.0162,  ..., -0.0550,  0.0287,  0.0143],
        ...,
        [-0.0406, -0.0324,  0.0329,  ...,  0.0080, -0.0130, -0.0109],
        [-0.0100, -0.0422,  0.0076,  ..., -0.0117, -0.0102,  0.0500],
        [ 0.0027,  0.0210,  0.0457,  ..., -0.0049, -0.0600, -0.0109]],
       requires_grad=True)
torch.float32
Size of the model before quantization
Size (KB): 251.618
Accuracy of the model before quantization: 


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 171.20it/s]

test acc 0.4745


# Quantization

- We select layers to quantize (like Linear layers)
- Not all layers are compatible for dynamic quantization.
- The weights of these layers are quantized to int8 ahead of time
- The model continues to take float32 inputs

1. **Input Handling**:
   - When float32 input hits a quantized layer
   - The input is temporarily quantized to int8 just for the matrix multiplication
   - This is done because int8 × int8 is much faster than float32 × float32 on CPU
   - This temporary quantization uses dynamic scaling factors calculated on-the-fly

In [6]:
print("applying post training dynamic quantization on model")
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

applying post training dynamic quantization on model


In [7]:
print("model summary")
print(quantized_model)
print('Weights after quantization')
print(torch.int_repr(quantized_model.fc1.weight()))
print('Size of the model after quantization')
q_size = print_size_of_model(quantized_model)
print('Accuracy of the model after quantization: ')
start = time.time()
q_acc = evaluate(quantized_model, test_dl)
end = time.time()
q_time = end-start

model summary
Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): DynamicQuantizedLinear(in_features=400, out_features=120, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (fc2): DynamicQuantizedLinear(in_features=120, out_features=84, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (fc3): DynamicQuantizedLinear(in_features=84, out_features=10, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)
Weights after quantization
tensor([[ 10, -42, -39,  ...,  -2, -14,  17],
        [-56,  15,   9,  ...,  25, -33,  17],
        [ -8,  24,  15,  ..., -51,  27,  13],
        ...,
        [-38, -30,  31,  ...,   7, -12, -10],
        [ -9, -39,   7,  ..., -11, -10,  47],
        [  3,  20,  43,  ...,  -5, -56, -10]], dtype=torch.int8)
Size of the model after quantization
Size (KB): 76.834
Accuracy of the model after qua

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:02<00:00, 117.38it/s]

test acc 0.4753




2. **During Layer Computation**:
   ```
   float32 input → temporary int8 → matrix multiply with int8 weights → float32 output
   ```

3. **Key Point**: We don't permanently quantize activations in dynamic quantization. The "dynamic" in dynamic quantization refers to:
   - The temporary quantization of inputs during computation
   - The scaling factors being calculated dynamically at runtime
   - No permanent storage of quantized activations

Dynamic quantization only permanently quantizes weights. The temporary input quantization is just an implementation detail to make matrix multiplication faster, and the outputs are immediately converted back to float32.

In [8]:
print('Original weights: ')
print(model.fc1.weight)
print('')
print('Dequantized weights: ')
print(torch.dequantize(quantized_model.fc1.weight()))
print('')

print(f"{'Prec':<6} | {'Accuracy':<8} | {'Model Size':<10} | {'Time Taken'}")
print(f"{'FP32':<6} | {o_acc:<8.4f} | {o_size:<10.3f} | {o_time:<.6f}")
print(f"{'INT8':<6} | {q_acc:<8.4f} | {q_size:<10.3f} | {q_time:<.6f}")


Original weights: 
Parameter containing:
tensor([[ 0.0104, -0.0451, -0.0420,  ..., -0.0018, -0.0146,  0.0187],
        [-0.0603,  0.0163,  0.0094,  ...,  0.0270, -0.0356,  0.0184],
        [-0.0088,  0.0259,  0.0162,  ..., -0.0550,  0.0287,  0.0143],
        ...,
        [-0.0406, -0.0324,  0.0329,  ...,  0.0080, -0.0130, -0.0109],
        [-0.0100, -0.0422,  0.0076,  ..., -0.0117, -0.0102,  0.0500],
        [ 0.0027,  0.0210,  0.0457,  ..., -0.0049, -0.0600, -0.0109]],
       requires_grad=True)

Dequantized weights: 
tensor([[ 0.0107, -0.0451, -0.0418,  ..., -0.0021, -0.0150,  0.0182],
        [-0.0601,  0.0161,  0.0097,  ...,  0.0268, -0.0354,  0.0182],
        [-0.0086,  0.0257,  0.0161,  ..., -0.0547,  0.0290,  0.0139],
        ...,
        [-0.0408, -0.0322,  0.0333,  ...,  0.0075, -0.0129, -0.0107],
        [-0.0097, -0.0418,  0.0075,  ..., -0.0118, -0.0107,  0.0504],
        [ 0.0032,  0.0215,  0.0461,  ..., -0.0054, -0.0601, -0.0107]])

Prec   | Accuracy | Model Size | Time Ta